In [1]:
# #once per cluster
# s3helper.open_bucket('mas-dse-open',region='us-west-2')

# info_files=s3helper.ls('Weather/Info')
# Without_path=[x[13:] for x in info_files]


# !mkdir /mnt/workspace/WeatherInfo

# for i in range(len(info_files)-1):   # Last entry is the directory name
#     _from=info_files[i]
#     _to='/mnt/workspace/WeatherInfo/'+Without_path[i]
#     print('From %s to %s'%(_from,_to))
#     s3helper.s3_to_local(_from,_to)
    
# s3helper.open_bucket('dse-weather-west-2', region="us-west-2")
# s3helper.ls('')
# s3helper.s3_to_hdfs("weather.parquet", "/tmp/weather.parquet")
# s3helper.s3_to_hdfs("info/stations.parquet", "/tmp/stations.parquet")
# print(s3helper.ls_hdfs("/tmp"))

In [1]:
import os
os.getcwd()
os.chdir("/mnt/workspace/Public-DSC291/notebooks/Section2-PCA/Full_Dataset_analysis")
os.getcwd()

from Startup import *

finished standard imports
dict_items([('spark.executor.cores', '1'), ('spark.executor.memory', '3g'), ('spark.cores.max', '4'), ('spark.logConf', 'True'), ('spark.app.name', 'Weather_PCA'), ('spark.default.parallelism', '10')])
started SparkContext and SQLContext in 13.50 seconds
loaded weather.parquet in 7.75 seconds
loaded stations.parquet in 0.17 seconds
registered dataframes as tables in 0.24 seconds


In [2]:
%%time
query="""
SELECT stations.ID, stations.elevation, stations.state, stations.latitude, stations.longitude,
        weather.Station, weather.Measurement, weather.Year, weather.Values
FROM stations
LEFT JOIN weather
ON stations.ID = weather.Station
WHERE stations.state='CA' AND weather.Measurement='PRCP'
"""

print(query)
CA_weather_df=sqlContext.sql(query)
# CA_weather_pdf=CA_weather_df.toPandas()
# sqlContext.registerDataFrameAsTable(CA_weather_df,'CA_weather_df')
# # CA_weather_df.show()
# CA_weather_pdf.head()


SELECT stations.ID, stations.elevation, stations.state, stations.latitude, stations.longitude,
        weather.Station, weather.Measurement, weather.Year, weather.Values
FROM stations
LEFT JOIN weather
ON stations.ID = weather.Station
WHERE stations.state='CA' AND weather.Measurement='PRCP'

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 79.2 ms


In [7]:
print('CA has %d rows'%(CA_weather_df.count()))
print(CA_weather_df.columns)

CA has 51948 rows
['ID', 'elevation', 'state', 'latitude', 'longitude', 'Station', 'Measurement', 'Year', 'Values']


In [4]:
CA_weather_df.show(2,truncate= True)

+-----------+---------+-----+--------+---------+-----------+-----------+----+--------------------+
|         ID|elevation|state|latitude|longitude|    Station|Measurement|Year|              Values|
+-----------+---------+-----+--------+---------+-----------+-----------+----+--------------------+
|USC00042239|   1414.3|   CA| 32.9897|-116.5872|USC00042239|       PRCP|1899|[00 7E 00 7E 0C 5...|
|USC00042239|   1414.3|   CA| 32.9897|-116.5872|USC00042239|       PRCP|1900|[00 00 00 00 84 5...|
+-----------+---------+-----+--------+---------+-----------+-----------+----+--------------------+
only showing top 2 rows



In [10]:
CA_weather_df.describe('latitude').show()

+-------+-----------------+
|summary|         latitude|
+-------+-----------------+
|  count|            51948|
|   mean|37.01589303149312|
| stddev|2.557845093758886|
|    min|          32.5667|
|    max|             42.0|
+-------+-----------------+



In [23]:
CA_weather_df.filter(CA_weather_df.Year > 1960)

DataFrame[ID: string, elevation: double, state: string, latitude: double, longitude: double, Station: string, Measurement: string, Year: int, Values: binary]

In [ ]:
CA_weather_df.

In [14]:
# CA_weather_df.groupby('Year').agg({'Values': 'mean'}).show()

In [24]:
rdd0=CA_weather_df.rdd.map(lambda row: unpackArray(row['Values'],np.float16))

In [25]:
rdd0.take(3)

[array([             nan,              nan,   3.23000000e+02,
          7.40000000e+01,              nan,              nan,
                     nan,   1.24000000e+02,   2.50000000e+01,
                     nan,   9.14000000e+02,   2.54000000e+02,
          6.10000000e+01,              nan,              nan,
                     nan,              nan,              nan,
                     nan,              nan,              nan,
                     nan,              nan,              nan,
                     nan,              nan,              nan,
                     nan,              nan,              nan,
                     nan,   0.00000000e+00,   4.30000000e+01,
          4.80000000e+01,   1.50000000e+01,   3.60000000e+01,
          3.00000000e+01,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
        

In [26]:
# UnDef=np.array(rdd0.map(lambda row:sum(np.isnan(row))).sample(False,0.01).collect())
# flat=rdd0.flatMap(lambda v:list(v)).filter(lambda x: not np.isnan(x)).cache()

In [28]:
rdd0.map(lambda x: )

SyntaxError: invalid syntax (<ipython-input-28-7e86d7d72f82>, line 1)